# Transformer training

In [1]:
%load_ext autoreload
%autoreload 2

from transformer_model import *
import nltk
import sys
sys.path.append("../../..")

In [2]:
#import statapp.transformer.common.get_positional_encodings
#from statapp.common.preprocessing import load_data, encode_data, split_into_X_y
def load_data(path, sample=1, split_on=" "):
    """Load a text dataset, optionally sampling the data rounding at some splitting string.
    
    Args:
        path (str): Path to a text file.
        sample (float): (approximate) Proportion of the dataset to load. 1 loads everything.
            Default: 1.
        split_on (str): String to split dataset on, e.g. "\n".
            When sampling, the dataset will be cut at the previous split_on substring.
            Ignored if sample==1.
            If None, samples the dataset by cutting at the previous character.
            Default: " ".
    
    Returns:
        string: Dataset.
    """
    with open(path, "r", encoding="utf-8") as file:
        text = file.read()
    
    if sample != 1:
        text = text[:int(len(text)*sample)]
        if split_on is not None:
            text = text[:-text[::-1].find(split_on)-1]
    
    return text

Preprocessing maison bien moche pour le moment... ^^

In [3]:
text = load_data("data/fr.train.top1M.txt")

In [4]:
tokens = nltk.word_tokenize(text[:1000000])

In [5]:
len(tokens)

193627

In [6]:
vocab = list(set(tokens))

In [7]:
vocab_size = len(vocab)

In [8]:
vocab_numbers = dict(zip(vocab, range(0,len(vocab))))

In [9]:
tokens_numbers = np.array([vocab_numbers[tokens[i]] for i in range(len(tokens))])

In [10]:
tokens_numbers_sequences = np.array([ tokens_numbers[i:i+vector_size] for i in range(len(tokens_numbers)-vector_size)])

In [11]:
tokens_numbers_sequences = torch.tensor(tokens_numbers_sequences , dtype=torch.int64)

In [12]:
nb_sequences =  tokens_numbers_sequences.shape[0]

In [13]:
LMtransformer = Transformer(vocab_size, Decoder(MultiHeadAttention(nb_heads, head_size, vector_size), FeedforwardNetwork(vector_size, ffn_hidden_size)))

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(LMtransformer.parameters(), lr=0.001)

In [17]:
def train_model(nb_epochs, batch_size):
    
    LMtransformer.train()
    
    for epoch in range(nb_epochs):
        
        running_loss = 0
        
        randperm = torch.randperm(nb_sequences)
        randperm = randperm[:(nb_sequences//batch_size)*batch_size]
        batchs_indices = randperm.reshape(nb_sequences//batch_size, batch_size)
        
        for batch_indices in batchs_indices:
            
            batch = tokens_numbers_sequences[batch_indices]
            optimizer.zero_grad()
            output = LMtransformer(batch)
            loss = criterion(output[:][:-1], batch[:][1:])

            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 2000 == 1999:    # print every 2000 mini-batches
                print('[%d, %5d] loss: %.3f' %
                      (epoch + 1, i + 1, running_loss / 2000))
                running_loss = 0.

In [19]:
train_model(1,1)

AttributeError: 'LayerNorm' object has no attribute 'dim'